In [9]:
import pickle
import pandas as pd
import os
import torch

import betfairlightweight
from betfairlightweight import filters
from datetime import datetime
from datetime import timedelta
from dateutil import tz
import math
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
import fasttrack as ft
import importlib
import featurecreations

# Import libraries for logging in
from flumine import Flumine, clients
from flumine import BaseStrategy 
from flumine.order.trade import Trade
from flumine.order.order import LimitOrder, OrderStatus
from flumine.markets.market import Market
from betfairlightweight.resources import MarketBook
from flumine.streams.datastream import DataStream
import logging 
import re
from ft_sec_key import SECKEY
from nltk.tokenize import regexp_tokenize

In [10]:
seckey = SECKEY
greys = ft.Fasttrack(seckey)
today = datetime.today().strftime('%Y-%m-%d')
ahead3 = (datetime.today()).strftime('%Y-%m-%d')

race_details, dog_results = greys.getBasicFormat(today, dt_end=ahead3)

race_details = race_details[['@id', 'RaceNum', 'Distance', 'Date','RaceTime', 'RaceName', 'RaceGrade', 'Track', 'State']]
race_details['RaceId'] = race_details['@id']
dog_results = dog_results[['@id', 'RaceBox', 'DogName', 'RaceId', 'DogGrade']]
dog_results['DogId'] = dog_results['@id']
dog_results['box'] = dog_results['RaceBox']

prediction_df = pd.merge(dog_results,race_details, how='left', on='RaceId')
prediction_df.DogName = prediction_df.DogName.apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip())

# x = featurecreations.generate_prediction_dataframe(r"./DATA/all-results-db.npy", prediction_df)

# x.to_pickle(f'prediction_input {today}.npy')

Valid Security Key
Getting meets for each date ..


100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


Getting dog lineups ..


100%|██████████| 15/15 [00:38<00:00,  2.57s/it]


In [3]:
prediction_df

,@id_x,RaceBox,DogName,RaceId,DogGrade,DogId,box,@id_y,RaceNum,Distance,Date,RaceTime,RaceName,RaceGrade,Track,State
0,745076702,1,SIPPY CUP EMU,947405564,M,745076702,1,947405564,1,460m,28 Sep 23,03:04PM,SPORTSBET GREEN TICKS,Tier 3 - Maiden,Warragul,VIC
1,788011385,2,FINANCIER,947405564,M,788011385,2,947405564,1,460m,28 Sep 23,03:04PM,SPORTSBET GREEN TICKS,Tier 3 - Maiden,Warragul,VIC
2,772515956,3,POT OF GOLD,947405564,M,772515956,3,947405564,1,460m,28 Sep 23,03:04PM,SPORTSBET GREEN TICKS,Tier 3 - Maiden,Warragul,VIC
3,694776866,4,SIR SUGAR,947405564,M,694776866,4,947405564,1,460m,28 Sep 23,03:04PM,SPORTSBET GREEN TICKS,Tier 3 - Maiden,Warragul,VIC
4,640324597,5,TAKE A CARNATION,947405564,M,640324597,5,947405564,1,460m,28 Sep 23,03:04PM,SPORTSBET GREEN TICKS,Tier 3 - Maiden,Warragul,VIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,782171737,4,ME SPOOK,946804852,6,782171737,4,946804852,11,297m,28 Sep 23,10:55PM,PBD 1-3 WINS LADBROKES MATES MODE,Grade 5,Dapto,NSW
1461,716345393,5,SNOWY ETHICS,946804852,5,716345393,5,946804852,11,297m,28 Sep 23,10:55PM,PBD 1-3 WINS LADBROKES MATES MODE,Grade 5,Dapto,NSW
1462,650078101,6,TIAS BABE,946804852,6,650078101,6,946804852,11,297m,28 Sep 23,10:55PM,PBD 1-3 WINS LADBROKES MATES MODE,Grade 5,Dapto,NSW
1463,782171232,7,FLAMING SONIA,946804852,7,782171232,7,946804852,11,297m,28 Sep 23,10:55PM,PBD 1-3 WINS LADBROKES MATES MODE,Grade 5,Dapto,NSW


In [4]:
prediction_df.columns

Index(['@id_x', 'RaceBox', 'DogName', 'RaceId', 'DogGrade', 'DogId', 'box',
       '@id_y', 'RaceNum', 'Distance', 'Date', 'RaceTime', 'RaceName',
       'RaceGrade', 'Track', 'State'],
      dtype='object')

In [5]:
prediction_df['race datetime']  =  pd.to_datetime(prediction_df.apply(lambda x: f"{x['Date']} {x['RaceTime']}",axis=1))

In [6]:
prediction_df['race datetime']

0      2023-09-28 15:04:00
1      2023-09-28 15:04:00
2      2023-09-28 15:04:00
3      2023-09-28 15:04:00
4      2023-09-28 15:04:00
               ...        
1460   2023-09-28 22:55:00
1461   2023-09-28 22:55:00
1462   2023-09-28 22:55:00
1463   2023-09-28 22:55:00
1464   2023-09-28 22:55:00
Name: race datetime, Length: 1465, dtype: datetime64[ns]

In [7]:
prediction_df.Track.value_counts()

Ipswich          116
Warrnambool      112
Healesville      111
Northam          111
Warragul         110
Christchurch     106
Sandown Park     101
Waikato           98
Mount Gambier     98
Hobart            93
Angle Park        87
Gunnedah          83
Casino            83
Dapto             83
Albion Park       73
Name: Track, dtype: int64

In [8]:
# Credentials to login and logging in 
trading = betfairlightweight.APIClient('nickbarlow@live.com.au','un6/chxe!N!?adsp',app_key='JFWqJHqB4Akfi5hK')
client = clients.BetfairClient(trading, interactive_login=True)

# Login
framework = Flumine(client=client)

trading.login_interactive()

racing_filter=betfairlightweight.filters.market_filter(
        event_type_ids=["4339"], # Greyhounds
        market_countries=["AU", "NZ"], # Australia
        market_type_codes=["WIN"], # Win Markets
    )

results = trading.betting.list_events(racing_filter)

for i in results:
    print(i.event.id, i.event.name, i.market_count)

results = trading.betting.list_venues(racing_filter)
venues = [v.venue for v in results]
for i in results:
    print(i, i.venue,i.market_count)

results_list = []

for v in venues:
    racing_filter=betfairlightweight.filters.market_filter(
        event_type_ids=["4339"], # Greyhounds
        market_countries=["AU", "NZ"], # Australia
        market_type_codes=["WIN"], # Win Markets
        venues=[v]
    )

    results = trading.betting.list_market_catalogue(
            market_projection=[
                "RUNNER_DESCRIPTION", 
                "RUNNER_METADATA", 
                "COMPETITION", 
                "EVENT", 
                "EVENT_TYPE", 
                "MARKET_DESCRIPTION", 
                "MARKET_START_TIME",
            ],
            filter=racing_filter,
            max_results=110,
        )

    results_list.extend(results)

df_list = []

box_change_list = []
pattern1 = r'(?<=<br>Dog ).+?(?= starts)'
pattern2 = r"(?<=\bbox no. )(\w+)"

for i in results_list:
    print(f"{i.market_id,i.market_name,i.market_start_time.hour, i.market_start_time.minute, i.event.venue,  i.description.market_type} ")
    market_id = i.market_id
    race_num = int(re.sub("[^0-9]", "", i.market_name.split(' ',1)[0] ))
    track =  i.event.venue
    dist = i.market_name.split(' ',2)[0]
    print(race_num)
    if i.description.clarifications:
        print(i.description.clarifications.replace("<br> Dog","<br>Dog"))
        box_change_list.append((track, race_num,i.description.clarifications.replace("<br> Dog","<br>Dog")))

    for dog in i.runners:
        print(f"id = {dog.selection_id}, name = {dog.runner_name.split(' ',1)[1].upper()}")
        df_list.append([market_id, track, dist, race_num, dog.selection_id,dog.runner_name.split('.',1)[0].upper(), dog.runner_name.split(' ',1)[1].upper()])
df = pd.DataFrame(data = df_list, columns=['market_id', 'track', 'dist', 'race_num', 'runner_id','box_bf', 'runnner_name'])

32669152 Warragul (AUS) 28th Sep 12
32669473 Cambridge (NZL) 28th Sep 12
32669217 Mount Gambier (AUS) 28th Sep 12
32669110 Ipswich (AUS) 28th Sep 12
32669350 Sandown Park (AUS) 28th Sep 12
32669095 Casino (AUS) 28th Sep 10
32669111 Warrnambool (AUS) 28th Sep 12
32669125 Albion Park (AUS) 28th Sep 10
32669214 Healesville (AUS) 28th Sep 12
VenueResult Cambridge 12
VenueResult Healesville 12
VenueResult Casino 10
VenueResult Mount Gambier 12
VenueResult Ipswich 12
VenueResult Warragul 12
VenueResult Warrnambool 12
VenueResult Sandown Park 12
VenueResult Albion Park 10
('1.218899722', 'R1 375m C0', 23, 31, 'Cambridge', 'WIN') 
1
id = 61070226, name = HIGHVIEW SPOT
id = 61070227, name = ZIPPING ANGIE
id = 61070228, name = LORD ONYX
id = 59147438, name = CENTRAL DIME
id = 61070229, name = SPECIAL LIGHT
id = 60230754, name = GOLD CARD
id = 58371925, name = FAWN OVER YOU
('1.218899724', 'R2 457m C0', 23, 49, 'Cambridge', 'WIN') 
2
id = 61070231, name = THRILLING VOLT
id = 60780069, name = THRI

In [50]:
tracks_FT = prediction_df.Track.unique()
tracks_FT

array(['Traralgon', 'Geelong', 'Healesville', 'Bendigo', 'Wanganui',
       'The Gardens', 'Richmond', 'Wagga Wagga', 'Christchurch',
       'Goulburn', 'Ipswich', 'Rockhampton', 'Cannington',
       'Murray Bridge'], dtype=object)

In [5]:
box_change_list

[('Ipswich',
  7,
  '<additionalinfo><br>Box changes:<br>Dog 9. Black Monaro starts from box no. 5<br><br>'),
 ('Ipswich',
  8,
  '<additionalinfo><br>Box changes:<br>Dog 10. Woosnam starts from box no. 2<br><br>'),
 ('Townsville',
  2,
  '<additionalinfo><br>Box changes:<br>Dog 9. Mr Gledhill starts from box no. 3<br><br>'),
 ('Townsville',
  5,
  '<additionalinfo><br>Box changes:<br>Dog 9. Dam Slipperiest starts from box no. 5<br>Dog 10. Denim starts from box no. 7<br><br>\n\n'),
 ('Townsville',
  12,
  '<additionalinfo><br>Box changes:<br>Dog 9. Ryans Pick starts from box no. 2<br><br>'),
 ('Gosford',
  1,
  '<additionalinfo><br>Box changes:<br>Dog 9. Butterwick starts from box no. 6<br><br>\n\n'),
 ('Gosford',
  4,
  '<additionalinfo><br>Box changes:<br>Dog 9. Jungle George starts from box no. 4<br><br>'),
 ('Gosford',
  11,
  '<additionalinfo><br>Box changes:<br>Dog 9. Mickey Roy starts from box no. 2<br><br>'),
 ('Cannington',
  5,
  '<additionalinfo><br>Box changes:<br>Dog 9. Jo

In [52]:
box_change_list[0][2].split('<br>')

['<additionalinfo>',
 'Box changes:',
 'Dog 9. Bill Buster starts from box no. 4',
 'Dog 10. Opawa Stream starts from box no. 6',
 '',
 '\n\n']

In [53]:
races_ft_df = df[['track','race_num', 'box_bf']].copy()
races_ft_df.columns = ['Track', 'RaceNum', 'RaceBox']
races_ft_df.replace('Wagga','Wagga Wagga',inplace = True)
races_ft_df.replace('Manawatu','Palmerston North',inplace = True)
races_ft_df.replace('Manukau','Waikato',inplace = True)
races_ft_df.replace('Addington','Christchurch',inplace = True)
races_ft_df.replace('Hatrick','Wanganui',inplace = True)


if 'Richmond Straight' in tracks_FT:
    races_ft_df.replace('Richmond','Richmond Straight',inplace = True)
    print("replacing richmond")

# 

In [54]:
races_ft_df

,Track,RaceNum,RaceBox
0,Christchurch,1,1
1,Christchurch,1,2
2,Christchurch,1,3
3,Christchurch,1,5
4,Christchurch,1,7
...,...,...,...
1111,Richmond,12,4
1112,Richmond,12,5
1113,Richmond,12,6
1114,Richmond,12,7


In [55]:
races_ft_df.RaceBox = pd.to_numeric(races_ft_df.RaceBox)

In [56]:
# box_change_df.string[8]

In [57]:
box_change_df = pd.DataFrame.from_records(box_change_list, columns = ['Track', 'RaceNum', 'string'])
#box_change_df.drop(index=0, inplace=True)
box_change_df.replace('Wagga','Wagga Wagga',inplace = True)
box_change_df.replace('Manawatu','Palmerston North',inplace = True)
box_change_df.replace('Manukau','Waikato',inplace = True)
box_change_df.replace('Addington','Christchurch',inplace = True)
box_change_df = box_change_df[~(box_change_df['string']=='  ')]
box_change_df['runner_name'] = box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern1)[0])
box_change_df['runner_name_2'] = box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern1)[-1])
box_change_df

,Track,RaceNum,string,runner_name,runner_name_2
0,Christchurch,1,<additionalinfo><br>Box changes:<br>Dog 9. Bil...,9. Bill Buster,10. Opawa Stream
1,Healesville,2,<additionalinfo><br>Box changes:<br>Dog 10. Mt...,10. Mt View Bentley,10. Mt View Bentley
2,Healesville,11,<additionalinfo><br>Box changes:<br>Dog 10. Mi...,10. Milo Rose,10. Milo Rose
3,Goulburn,2,<additionalinfo><br>Box changes:<br>Dog 9. Bla...,9. Blackpool Buddy,9. Blackpool Buddy
4,Goulburn,4,<additionalinfo><br>Box changes:<br>Dog 9. Zip...,9. Zipping Pip,9. Zipping Pip
5,Goulburn,8,<additionalinfo><br>Box changes:<br>Dog 9. Win...,9. Winlock Flowers,9. Winlock Flowers
6,Ipswich,1,<additionalinfo><br>Box changes:<br>Dog 9. Est...,9. Estonian Lady,10. Mister Mischief
7,Hatrick,1,<additionalinfo><br>Box changes:<br>Dog 9. Big...,9. Big Time Shred,9. Big Time Shred
8,Hatrick,10,<additionalinfo><br>Box changes:<br>Dog 9. All...,9. Allegro Bandit,9. Allegro Bandit
9,The Gardens,11,<additionalinfo><br>Box changes:<br>Dog 9. Elu...,9. Elusive Catch,9. Elusive Catch


In [67]:
box_change_df = pd.DataFrame.from_records(box_change_list, columns = ['Track', 'RaceNum', 'string'])
#box_change_df.drop(index=0, inplace=True)
box_change_df = box_change_df[~(box_change_df['string']==' ')]
box_change_df.replace('Wagga','Wagga Wagga',inplace = True)
box_change_df.replace('Manawatu','Palmerston North',inplace = True)
box_change_df.replace('Manukau','Waikato',inplace = True)
box_change_df.replace('Addington','Christchurch',inplace = True)
box_change_df.replace('Hatrick','Wanganui',inplace = True)
box_change_df = box_change_df[~(box_change_df['string']=='  ')]
box_change_df['runner_name'] = box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern1)[0])
box_change_df['runner_name'] = box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern1)[0])
box_change_df['runner_name_2'] = box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern1)[-1])

box_change_df['runner_number'] = box_change_df['runner_name'].apply(lambda x: x[:(x.find(" ") - 1)].upper())
box_change_df['runner_name'] = box_change_df['runner_name'].apply(lambda x: x[(x.find(" ") + 1):].upper())
box_change_df['RaceBox'] = pd.to_numeric(box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern2)[0]))
prediction_df.RaceNum = pd.to_numeric(prediction_df.RaceNum)
prediction_df.RaceBox = pd.to_numeric(prediction_df.RaceBox)
prediction_df = prediction_df.reset_index(drop=True).set_index(['Track', 'RaceNum', 'RaceBox'])
prediction_df = prediction_df.drop(index=list(box_change_df[['Track','RaceNum','RaceBox']].itertuples(index=False)), errors='ignore').reset_index()
prediction_df = prediction_df.merge(races_ft_df, on=['Track', 'RaceNum', 'RaceBox'])
merged = prediction_df.merge(how='left', right=box_change_df, right_on=['runner_name', 'RaceNum'], left_on=['DogName', 'RaceNum'])
merged['Box'] = merged.apply(lambda x: int(x['box']) if np.isnan(x['RaceBox_y']) else int(x['RaceBox_y']), axis =1)
merged['race_time'] = pd.to_datetime(merged['RaceTime'])
new_form = merged[merged['Box']<=8][['Track_x','State','DogId','DogName','DogGrade', 'RaceId','Distance', 'Date', 'Box','race_time','RaceNum']].rename(columns={'Track_x':'Track', 'RaceNum':'race_num'})

In [68]:
new_form

,Track,State,DogId,DogName,DogGrade,RaceId,Distance,Date,Box,race_time,race_num
0,Traralgon,VIC,709053782,REMO ADALLI,M,892806553,395m,21 Apr 23,1,2023-04-21 18:52:00,1
1,Traralgon,VIC,684480231,DARLINGTON MISS,M,892806553,395m,21 Apr 23,2,2023-04-21 18:52:00,1
2,Traralgon,VIC,714641938,SWOOPING SWAN,M,892806553,395m,21 Apr 23,3,2023-04-21 18:52:00,1
3,Traralgon,VIC,697264252,INDY,M,892806553,395m,21 Apr 23,4,2023-04-21 18:52:00,1
4,Traralgon,VIC,570983688,VICTORY BAZOOKA,M,892806553,395m,21 Apr 23,5,2023-04-21 18:52:00,1
...,...,...,...,...,...,...,...,...,...,...,...
1111,Murray Bridge,SA,575326859,MANUKA BOY,5,889068557,395m,21 Apr 23,1,2023-04-21 18:04:00,10
1112,Murray Bridge,SA,566432154,TERRIFIC MOLLY,5,889068557,395m,21 Apr 23,2,2023-04-21 18:04:00,10
1113,Murray Bridge,SA,640219897,KANJI SERVICE,5,889068557,395m,21 Apr 23,4,2023-04-21 18:04:00,10
1114,Murray Bridge,SA,620539155,PARAMOUNT ROSE,6,889068557,395m,21 Apr 23,5,2023-04-21 18:04:00,10


In [69]:
box_change_df

,Track,RaceNum,string,runner_name,runner_name_2,runner_number,RaceBox
0,Christchurch,1,<additionalinfo><br>Box changes:<br>Dog 9. Bil...,BILL BUSTER,10. Opawa Stream,9,4
1,Healesville,2,<additionalinfo><br>Box changes:<br>Dog 10. Mt...,MT VIEW BENTLEY,10. Mt View Bentley,10,5
2,Healesville,11,<additionalinfo><br>Box changes:<br>Dog 10. Mi...,MILO ROSE,10. Milo Rose,10,2
3,Goulburn,2,<additionalinfo><br>Box changes:<br>Dog 9. Bla...,BLACKPOOL BUDDY,9. Blackpool Buddy,9,2
4,Goulburn,4,<additionalinfo><br>Box changes:<br>Dog 9. Zip...,ZIPPING PIP,9. Zipping Pip,9,7
5,Goulburn,8,<additionalinfo><br>Box changes:<br>Dog 9. Win...,WINLOCK FLOWERS,9. Winlock Flowers,9,6
6,Ipswich,1,<additionalinfo><br>Box changes:<br>Dog 9. Est...,ESTONIAN LADY,10. Mister Mischief,9,4
7,Wanganui,1,<additionalinfo><br>Box changes:<br>Dog 9. Big...,BIG TIME SHRED,9. Big Time Shred,9,8
8,Wanganui,10,<additionalinfo><br>Box changes:<br>Dog 9. All...,ALLEGRO BANDIT,9. Allegro Bandit,9,2
9,The Gardens,11,<additionalinfo><br>Box changes:<br>Dog 9. Elu...,ELUSIVE CATCH,9. Elusive Catch,9,5


In [70]:
importlib.reload(featurecreations)
test_out = featurecreations.generate_prediction_dataframe_v2('results-df-merged-prices_cut.fth', new_form, simple=True)
print(f"input size = {len(test_out['stats'][0])}")

fin 1
['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']
input size = 20


In [71]:
new_form.Track.value_counts()

Healesville      96
Rockhampton      95
Bendigo          91
The Gardens      91
Richmond         91
Christchurch     90
Ipswich          90
Geelong          88
Wagga Wagga      84
Wanganui         80
Goulburn         78
Traralgon        76
Murray Bridge    59
Name: Track, dtype: int64

In [72]:
test_out.columns

Index(['dogid', 'dog_name', 'raceid', 'race_num', 'race_grade', 'date',
       'race_time', 'trackOHE', 'track_name', 'state', 'track_code', 'dist',
       'bfSP', 'runtime', 'prev_race', 'prev_race_date', 'prev_race_track',
       'place', 'margin', 'box', 'stats'],
      dtype='object')

In [73]:
test_out = test_out[test_out['raceid'].isin( new_form['RaceId'])]
with open(f"testing new outs simple {today}.npy", 'wb') as f:
    x = (test_out, ['speed_avg_1', 'speed_max_1', 'split_speed_avg_1', 'split_speed_max_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1'])
    pickle.dump(x, f)

In [74]:
len(test_out.stats.iloc[0])

20

In [75]:
['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_']
['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_']
['speed_avg_1', 'speed_max_1', 'split_speed_avg_1', 'split_speed_max_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1']

['speed_avg_1',
 'speed_max_1',
 'split_speed_avg_1',
 'split_speed_max_1',
 'split_margin_avg_1',
 'margin_avg_1',
 'first_out_avg_1',
 'post_change_avg_1',
 'races_1',
 'wins_1']